In [50]:
import proplot as pplt
import matplotlib.pyplot as plt
import os
import yaml
from tqdm import tqdm
import pandas as pd

In [51]:
records = []
for lr in [0.01]:
    dname = f"results_lr-{lr}"
    files = os.listdir(dname)
    for f in tqdm(files):
        with open(f"{dname}/{f}", 'r') as io:
            r = yaml.load(io, yaml.SafeLoader)
        scenario, seed = f.replace('.yaml', '').split('-')
        r['lr'] = lr
        r['scenario'] = scenario.split('_')[1]
        r['seed'] = seed.split('_')[1]
        records.append(r)

100%|██████████| 1201/1201 [00:07<00:00, 156.81it/s]


In [52]:
records = pd.DataFrame(records)
records = records.rename({'ate_error': 'bias'}, axis=1)
records["mean_population_ate"] = records["ate_estimate"] - records["bias"]
records.head()

,bias,ate_estimate,mse_loss,train_loss,lr,scenario,seed,mean_population_ate
0,0.985977,-1.616992,0.000002,0.010423,0.01,5,1153001,-2.602969
1,0.576850,-0.821197,0.000002,0.009768,0.01,3,1118001,-1.398047
2,1.715972,-0.880232,0.000001,0.010202,0.01,1,1059001,-2.596205
3,0.985402,-0.391670,0.000001,0.010027,0.01,3,1102001,-1.377072
4,1.450070,-1.059416,0.000001,0.008644,0.01,1,1053001,-2.509486


In [53]:
results = records.drop("seed", axis=1).groupby(["lr", "scenario"]).mean().reset_index()
results

,lr,scenario,bias,ate_estimate,mse_loss,train_loss,mean_population_ate
0,0.01,1,1.041736,-1.525682,0.000002,0.010062,-2.567417
1,0.01,2,1.041459,-0.357313,0.000002,0.010115,-1.398773
2,0.01,3,0.957643,-0.441129,0.000002,0.009728,-1.398773
3,0.01,4,0.970889,-1.595446,0.000004,0.011721,-2.566335
4,0.01,5,1.045989,-1.521428,0.000002,0.010045,-2.567417
5,0.01,6,0.416021,-2.151396,0.000002,0.009671,-2.567417


In [54]:
results.to_csv("benchmark-results-dragonnet.csv", index=False)